In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from models.create_dataset import create_dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
# lasso model
from epftoolbox.models import LEAR
from sklearn.linear_model import Lasso
import logging
import os
import time
# import MAE
from sklearn.metrics import mean_absolute_error


In [2]:

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
                    , handlers=[logging.FileHandler('LEAR_epf.log'), logging.StreamHandler()])
logger = logging.getLogger(__name__)


In [3]:
# load data
target_col = 'DK1_price'
df = create_dataset(target_col=target_col)

['NO_Solar', 'NO_Solar_lag_1', 'NO_Solar_lag_2', 'NO_Solar_lag_3', 'NO_Solar_lag_7']


In [4]:
df.head()

,DE_Solar,DK1_Solar,DK2_Solar,DK_Wind,NO_Wind,DE_Wind,SE_Wind,DK1_price,DE_price,DE_Solar_lag_1,...,DE_price_lag_2,DE_price_lag_3,DE_price_lag_7,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
DateTime,,,,,,,,,,,,,,,,,,,,,
2015-03-03 23:00:00,0.0,0.0,0.0,974.68,93.27,14102.28,921.48,24.44,23.31,0.0,...,18.00,11.76,29.62,0,1,0,0,0,0,0
2015-03-04 00:00:00,0.0,0.0,0.0,1080.81,105.96,15577.88,882.03,23.80,16.25,0.0,...,15.38,4.20,30.04,0,0,1,0,0,0,0
2015-03-04 01:00:00,0.0,0.0,0.0,1112.65,108.83,15496.93,856.06,23.38,15.37,0.0,...,15.10,-0.05,28.68,0,0,1,0,0,0,0
2015-03-04 02:00:00,0.0,0.0,0.0,1120.14,144.66,15667.33,840.29,23.62,15.50,0.0,...,13.00,-0.09,29.45,0,0,1,0,0,0,0
2015-03-04 03:00:00,0.0,0.0,0.0,1110.01,129.67,15806.69,848.01,24.43,18.04,0.0,...,12.00,-2.00,30.02,0,0,1,0,0,0,0


In [5]:
# split into X and y
y = df[target_col]
X = df.drop(target_col, axis=1)

In [6]:
# Pivot hourly index out to columns so index is only date
pivot_columns = [col for col in X.columns if not col.startswith('day_of_week')]
X = X.pivot_table(index=X.index.date, columns=X.index.hour, values=pivot_columns)
X = X.dropna()
# Some hours will only have 0 values, drop these columns (e.g. Solar)
X = X.loc[:, (X != 0).any(axis=0)]
# and some are 0 almost always, drop features with a MAD below threshold
X = X.loc[:, X.sub(X.median(axis=0), axis=1).abs().median(axis=0) > 0.01]

# apparently all variables need to be named Exogenous 1, 2, 3, etc.
X.columns = ['Exogenous ' + str(i) for i in range(1, X.shape[1] + 1)]


In [7]:
X.head()

,Exogenous 1,Exogenous 2,Exogenous 3,Exogenous 4,Exogenous 5,Exogenous 6,Exogenous 7,Exogenous 8,Exogenous 9,Exogenous 10,...,Exogenous 786,Exogenous 787,Exogenous 788,Exogenous 789,Exogenous 790,Exogenous 791,Exogenous 792,Exogenous 793,Exogenous 794,Exogenous 795
2015-03-04,1.61,135.51,2802.50,6947.57,10611.86,12958.45,13903.10,13794.82,12197.60,9112.27,...,1462.12,1404.33,1353.95,1371.81,1380.46,1323.80,1254.40,1193.39,1138.35,1110.53
2015-03-05,1.07,97.39,1746.14,4820.43,8451.17,11572.79,13332.19,13894.25,12971.14,10044.54,...,2411.63,2620.96,2812.71,3009.21,3170.08,3297.00,3377.08,3384.15,3349.22,3656.43
2015-03-06,4.82,182.01,2133.91,5133.71,7904.90,9958.42,11006.77,10527.40,9209.14,7107.95,...,2190.45,2069.54,1976.02,1931.76,1903.50,1897.43,1862.16,1802.85,1743.52,1493.72
2015-03-07,5.82,396.18,3934.54,8869.36,13397.02,16428.84,17644.92,17285.69,15063.98,11405.61,...,2407.09,2582.73,2717.26,2865.11,2974.29,3098.53,3219.09,3313.39,3383.36,3497.89
2015-03-08,5.35,407.54,4755.75,10486.90,15682.61,19291.68,20827.72,20570.41,18463.58,14432.91,...,4162.51,3996.05,3843.11,3685.99,3571.14,3452.32,3352.56,3254.99,3159.89,3188.75


In [8]:
# now y
# make y to dataframe first
y = y.rename('Price')
y = pd.DataFrame(y)
y = y.pivot_table(index=y.index.date, columns=y.index.hour, values=y.columns)
# join multiindex columns to one, price with hour number
y = y.dropna()
# rename to 'Price' for some reason for this toolbox to work


In [9]:
y.head()

Price                                                          \
DateTime       0      1      2      3      4      5      6      7      8    
2015-03-04  23.80  23.38  23.62  24.43  25.52  26.80  28.36  29.41  28.12   
2015-03-05  24.28  24.02  24.25  24.97  25.63  26.83  29.03  37.89  36.22   
2015-03-06  24.38  23.96  23.94  24.85  25.57  26.28  27.44  31.89  33.82   
2015-03-07  22.79  22.00  21.70  21.39  21.50  21.78  21.95  22.25  23.07   
2015-03-08  15.59  15.92  15.57  16.24  16.90  15.53  18.64  20.08  21.42   

                   ...                                                   \
DateTime       9   ...     14     15     16     17     18     19     20   
2015-03-04  26.99  ...  26.51  26.67  27.26  28.35  27.49  27.01  26.62   
2015-03-05  29.73  ...  33.91  36.25  45.71  50.91  42.20  27.10  26.71   
2015-03-06  32.02  ...  26.29  26.31  27.37  31.82  27.24  26.14  25.80   
2015-03-07  23.31  ...  22.09  22.73  23.85  24.42  24.24  23.26  22.29   
2015-03-08  21.42  ...  21.46  22.03  24.46  25.31  25.28  25.00  24.55   

                                 
DateTime       21     22     23  
2015-03-04  26.28  25.22  25.02  
2015-03-05  26.19  25.54  25.04  
2015-03-06  25.52  25.04  23.62  
2015-03-07  22.12  21.72  17.73  
2015-03-08  23.75  22.69  22.26  

[5 rows x 24 columns]

In [10]:


##### temporary start cut off as well to see if it works
#start_cutoff = pd.to_datetime('2019-01-01 00:00')
val_cutoff = pd.to_datetime('2020-07-01')
test_cutoff = pd.to_datetime('2021-01-01')
#X_train = X.loc[X.index >= start_cutoff & X.index < val_cutoff]
X_train = X.loc[X.index < val_cutoff]
X_val = X.loc[(X.index >= val_cutoff) & (X.index < test_cutoff)]
X_test = X.loc[X.index >= test_cutoff]
#y_train = y.loc[y.index < val_cutoff & y.index >= start_cutoff]
y_train = y.loc[y.index < val_cutoff]
y_val = y.loc[(y.index >= val_cutoff) & (y.index < test_cutoff)]
y_test = y.loc[y.index >= test_cutoff]

In [11]:
X

,Exogenous 1,Exogenous 2,Exogenous 3,Exogenous 4,Exogenous 5,Exogenous 6,Exogenous 7,Exogenous 8,Exogenous 9,Exogenous 10,...,Exogenous 786,Exogenous 787,Exogenous 788,Exogenous 789,Exogenous 790,Exogenous 791,Exogenous 792,Exogenous 793,Exogenous 794,Exogenous 795
2015-03-04,1.61,135.51,2802.50,6947.57,10611.86,12958.45,13903.10,13794.82,12197.60,9112.27,...,1462.12,1404.33,1353.95,1371.81,1380.46,1323.80,1254.40,1193.39,1138.35,1110.53
2015-03-05,1.07,97.39,1746.14,4820.43,8451.17,11572.79,13332.19,13894.25,12971.14,10044.54,...,2411.63,2620.96,2812.71,3009.21,3170.08,3297.00,3377.08,3384.15,3349.22,3656.43
2015-03-06,4.82,182.01,2133.91,5133.71,7904.90,9958.42,11006.77,10527.40,9209.14,7107.95,...,2190.45,2069.54,1976.02,1931.76,1903.50,1897.43,1862.16,1802.85,1743.52,1493.72
2015-03-07,5.82,396.18,3934.54,8869.36,13397.02,16428.84,17644.92,17285.69,15063.98,11405.61,...,2407.09,2582.73,2717.26,2865.11,2974.29,3098.53,3219.09,3313.39,3383.36,3497.89
2015-03-08,5.35,407.54,4755.75,10486.90,15682.61,19291.68,20827.72,20570.41,18463.58,14432.91,...,4162.51,3996.05,3843.11,3685.99,3571.14,3452.32,3352.56,3254.99,3159.89,3188.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,0.00,0.00,47.84,1750.31,4711.27,7346.73,8618.44,8043.63,6039.77,2752.80,...,3502.10,3406.35,3285.42,3157.04,3077.65,3025.82,2983.10,2933.27,2891.78,2900.04
2022-12-28,0.00,0.00,29.88,1097.06,2749.71,4287.79,5198.15,4939.23,3591.65,1809.27,...,2925.08,2996.64,3109.74,3266.29,3450.58,3642.46,3842.31,4030.38,4165.30,3778.74
2022-12-29,0.00,0.00,20.41,1451.16,3915.38,6092.62,7157.69,6731.00,5070.41,2314.00,...,4093.50,4261.85,4503.01,4728.38,4972.37,5203.29,5379.01,5490.52,5588.04,5576.82
2022-12-30,0.00,0.00,35.85,2514.91,6953.79,10357.79,11230.83,9424.72,6146.10,2476.54,...,5965.31,6113.25,6307.21,6326.51,6288.97,6213.99,6162.93,6168.45,6182.59,6356.44


In [12]:
# split into train and test
# set start time as first time in index where hour is 0
#
# start_time = X[X.index.hour == 0].index[0]
# test_cutoff = pd.to_datetime('2021-01-01 00:00')
# X = X[X.index >= start_time]
# y = y[y.index >= start_time]
#
# # subtract one hour
# X_train, X_test = X[X.index < test_cutoff], X[X.index >= test_cutoff]
# y_train, y_test = y[y.index < test_cutoff], y[y.index >= test_cutoff]

In [13]:
# split into train and validation
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

In [14]:
#start_time
#pd.to_datetime('2021-01-01 00:00')
y_test.head()

Price                                                          \
DateTime       0      1      2      3      4      5      6      7      8    
2021-01-01  48.19  44.68  42.92  40.39  40.20  39.63  40.09  41.27  44.88   
2021-01-02  42.43  41.09  40.00  37.55  39.00  42.09  44.96  48.45  52.00   
2021-01-03  41.59  37.21  35.18  24.65  23.85  24.05  24.80  24.97  25.65   
2021-01-04  24.05  23.88  24.03  24.58  25.84  28.21  30.85  49.20  50.02   
2021-01-05  27.45  27.08  27.14  27.79  29.81  35.78  53.64  60.00  60.12   

                   ...                                                   \
DateTime       9   ...     14     15     16     17     18     19     20   
2021-01-01  45.00  ...  46.59  52.99  60.26  60.61  60.36  57.40  53.86   
2021-01-02  52.00  ...  53.45  59.99  62.00  63.08  62.17  60.03  55.03   
2021-01-03  26.00  ...  26.50  28.05  28.53  28.17  27.92  27.56  27.04   
2021-01-04  55.40  ...  58.03  60.50  65.22  60.05  53.73  37.68  34.82   
2021-01-05  61.18  ...  62.51  64.26  65.60  60.13  54.66  40.78  39.60   

                                 
DateTime       21     22     23  
2021-01-01  53.45  49.72  46.69  
2021-01-02  52.25  48.45  46.11  
2021-01-03  26.18  24.92  24.66  
2021-01-04  30.95  27.35  27.81  
2021-01-05  32.06  29.21  29.83  

[5 rows x 24 columns]

In [15]:
# fit Lear model
calibration_window = 364 * 2  # 2 years
#day_range = pd.date_range(start=test_cutoff, end=y.index[-1], freq='D')
day_range = pd.date_range(start=pd.to_datetime('2022-01-01'), end=y.index[-1], freq='D')
preds = pd.DataFrame(index=y_test.index, columns=['pred'])
preds_done = pd.read_pickle(os.path.join(os.getcwd(), 'predictions', 'lear_preds.pkl'))

forecast = pd.DataFrame(index=X_test.index, columns=['h' + str(k) for k in range(24)])
forecast.index = pd.to_datetime(forecast.index)
preds_done = preds_done.dropna()
preds_done.index = pd.to_datetime(preds_done.index)
# Acess first element of list for each row in preds_done
for i, date in enumerate(preds_done.index):
    forecast.loc[date] = preds_done.iloc[i]['pred'][0]#forecast.loc[preds_done.index] = preds_done.values.reshape(-1, 1)

model = LEAR(calibration_window=calibration_window)

# rename to 'Price' for some reason for this toolbox to work
target_col = 'Price'



time_start = time.time()
for i, day in enumerate(day_range):
    if day.day == 1:
        logger.info(f'Predicting day {day} ({i+1}/{len(day_range)})')
        # save predictions
        preds.to_pickle(os.path.join(os.getcwd(), 'predictions', 'lear_preds.pkl'))
    # get train data
    X_train_date = X[X.index < day].values
    y_train_date = y[y.index < day].values

    X_test_date = X[(X.index >= day) & (X.index < day + pd.Timedelta(days=1))].values

    pred = model.recalibrate_predict(X_train_date, y_train_date, X_test_date)

    forecast.loc[day] = pred
    print(f'MAE: {mean_absolute_error(y_test[y_test.index == day.date()], pred)}')


2023-05-09 09:39:07,791 - __main__ - INFO - Predicting day 2022-01-01 00:00:00 (1/365)


MAE: 18.635865646837846
MAE: 11.453427625401636
MAE: 28.280955958713367
MAE: 46.715294190591614
MAE: 32.5433179802497
MAE: 60.403027956729886
MAE: 64.50351076748018
MAE: 13.853541266126916
MAE: 27.951152298459892
MAE: 95.99482623150793
MAE: 17.850644412721284
MAE: 20.774506379507088
MAE: 82.30703375423114
MAE: 32.08857824598258
MAE: 33.61905363661506
MAE: 26.975876196626
MAE: 18.742392905293087
MAE: 39.61473332342037
MAE: 45.55407141718064
MAE: 23.611082285206475
MAE: 18.736951603053686
MAE: 32.309135763982766
MAE: 16.069544093224767
MAE: 25.55130141296682
MAE: 33.17409047115891
MAE: 13.930303287038365
MAE: 50.85847515692985
MAE: 25.193640019029406
MAE: 22.294832102686097
MAE: 21.628385494716895


2023-05-09 10:31:15,572 - __main__ - INFO - Predicting day 2022-02-01 00:00:00 (32/365)


MAE: 14.484453067845385
MAE: 21.009366417715096
MAE: 14.19825825406267
MAE: 34.80346730760067
MAE: 17.735944939121406
MAE: 33.75239467407936
MAE: 17.082392017488175
MAE: 18.43298806265177
MAE: 22.619083799534266
MAE: 19.593989215853913
MAE: 9.855203255087785
MAE: 44.31091790742053
MAE: 20.570101099019677
MAE: 14.26468074051671
MAE: 15.306498505928923
MAE: 21.449478195964293
MAE: 19.959730142794843
MAE: 12.185319416641414
MAE: 12.30462012285471
MAE: 21.560903318128073
MAE: 12.442278915201447
MAE: 16.695130581376713
MAE: 8.549795322558671
MAE: 15.83663632677287
MAE: 17.478666750071444
MAE: 12.699487672990173
MAE: 35.69579661515948
MAE: 37.742176927552066


2023-05-09 11:16:14,914 - __main__ - INFO - Predicting day 2022-03-01 00:00:00 (60/365)


...

MAE: 37.91542622827353
MAE: 6.609065397760186
MAE: 23.182319516434507
MAE: 42.29850908661367
MAE: 33.35864701519101
MAE: 34.9782376231447
MAE: 65.36100157044832
MAE: 78.73960708594562
MAE: 43.147323015136266
MAE: 22.606946534767147
MAE: 29.730516823287644
MAE: 94.77545691517996
MAE: 126.66772825108508
MAE: 78.19731536677652
MAE: 52.5006132560232
MAE: 23.136050531103233
MAE: 19.068729509614723
MAE: 37.06853420207506
MAE: 37.577082733288236
MAE: 20.80604038319014
MAE: 12.134308106580084
MAE: 14.36934429233167
MAE: 10.22453538083795
MAE: 15.10003471890923
MAE: 6.664244910627018
MAE: 13.2098443286661
MAE: 8.384655522721237
MAE: 10.450828463492682
MAE: 8.053229624177433
MAE: 10.036811444937976
MAE: 8.552195630046842


2023-05-09 18:52:39,901 - __main__ - INFO - Predicting day 2022-11-01 00:00:00 (305/365)


MAE: 11.727209131412446
MAE: 8.94213987946098
MAE: 9.16778693383693
MAE: 6.820167759835388
MAE: 53.449848557114684
MAE: 33.899797973797725
MAE: 14.014118883712122
MAE: 18.351698044866946
MAE: 23.462522346988777
MAE: 42.50885589745953
MAE: 48.214192273852056
MAE: 83.37633977865028
MAE: 49.88897134919341
MAE: 25.822029786540565
MAE: 45.11049982976809
MAE: 56.63195396344556
MAE: 70.41567645453092
MAE: 30.826645426359818
MAE: 15.361610427251689
MAE: 73.4288345343573
MAE: 51.205703663255406
MAE: 47.910549854487726
MAE: 45.10918399571245
MAE: 32.46241087083644
MAE: 79.40994252309059
MAE: 74.41394751257589
MAE: 42.11610856271645
MAE: 20.939686564780946
MAE: 32.12343877526643
MAE: 139.74327007007753


2023-05-09 19:59:42,166 - __main__ - INFO - Predicting day 2022-12-01 00:00:00 (335/365)


MAE: 138.17269685891694
MAE: 121.9103992548159
MAE: 90.50474882882988
MAE: 67.54218645392244
MAE: 62.26223707611819
MAE: 67.28851563400359
MAE: 83.3680437459182
MAE: 51.64338592272302
MAE: 103.56433714631403
MAE: 127.83276932542174
MAE: 83.9121987995831
MAE: 91.75873237467836
MAE: 184.78620067351753
MAE: 175.95835730530325
MAE: 141.68320599711342
MAE: 97.0158870031979
MAE: 152.59656063644272
MAE: 82.59287526757574
MAE: 50.36333208756861
MAE: 57.0487086250616
MAE: 42.01338793296082
MAE: 30.608593613548425
MAE: 31.444520765509694
MAE: 19.46710200377997
MAE: 8.87997404840255
MAE: 12.156487437076889
MAE: 4.890262575577466
MAE: 15.31181689294526
MAE: 17.04731878750459
MAE: 5.100707511007139
MAE: 5.125732911077132
MAE: 15.700093997251694


In [17]:
# save preds
forecast.to_pickle(os.path.join(os.getcwd(), 'predictions', 'lear_preds.pkl'))

In [18]:
tmp = pd.read_pickle(os.path.join(os.getcwd(), 'predictions', 'lear_preds.pkl'))

In [21]:
forecast.to_pickle(os.path.join(os.getcwd(), 'predictions', 'lear_preds_2022.pkl'))

In [1]:
AAA = pd.read_pickle(os.path.join(os.getcwd(), 'predictions', 'lear_preds_2021.pkl'))

NameError: name 'pd' is not defined

In [2]:
import pandas as pd
import os

In [5]:
AAA = pd.read_pickle(os.path.join(os.getcwd(), 'predictions', 'lear_preds_2021.pkl'))
AAA1 = pd.read_pickle(os.path.join(os.getcwd(), 'predictions', 'lear_preds_2021_1.pkl'))

In [6]:
BBB = pd.read_pickle(os.path.join(os.getcwd(), 'predictions', 'lear_preds_2022.pkl'))

In [7]:
BBB[BBB.index.year == 2021] = AAA1.dropna()

In [9]:
BBB.to_pickle(os.path.join(os.getcwd(), 'predictions', 'lear_preds_all.pkl'))